In [1]:
import pymongo

In [2]:
import string
import random

In [3]:
url = "mongodb://10.1.0.32:27020"

In [4]:
client = pymongo.MongoClient(url)

In [5]:
def create_users(database, n_client=200):
    client = database['Client']
    items = client.find()
    id_start = 0
    
    if client.count_documents({}) != 0:
        id_start = items.sort([("id", pymongo.DESCENDING)])[0]['id'] + 1
    package = []
    
    for i in range(n_client):
        firstName = ''.join(random.choices(string.ascii_letters + string.digits, k=random.randint(5,25)))
        lastName = ''.join(random.choices(string.ascii_letters + string.digits, k=random.randint(5,25)))
        item = {"id": id_start, 
                "firstName": firstName, 
                "lastName": lastName}
        package.append(item)
        id_start += 1
        
    client.insert_many(package)

In [6]:
def create_drivers(database, n_client=200):
    driver = database['Driver']
    items = client.find()
    id_start = 0
    
    if driver.count_documents({}) != 0:
        id_start = items.sort([("id", pymongo.DESCENDING)])[0]['id'] + 1
    package = []
    
    for i in range(n_client):
        firstName = ''.join(random.choices(string.ascii_letters + string.digits, k=random.randint(5,25)))
        lastName = ''.join(random.choices(string.ascii_letters + string.digits, k=random.randint(5,25)))
        car = ''.join(random.choices(string.ascii_letters + string.digits, k=random.randint(5,25)))
        item = {"id": id_start, 
                "firstName": firstName, 
                "lastName": lastName, 
                "car": car, 
                "raiting": None}
        package.append(item)
        id_start += 1
        
    driver.insert_many(package)

In [7]:
def get_drivers_id(taxi):
    drivers = taxi['Driver']
    items_drivers = drivers.find()
    return [i['id'] for i in items_drivers]

In [8]:
def get_clients_id(taxi):
    clients = taxi['Client']
    items_clients = clients.find()
    return [i['id'] for i in items_clients]

In [9]:
def create_orders(taxi, london_address, n_count=50):
    orders = taxi['Orders']
    items_orders = orders.find()
    ids_clients = get_clients_id(taxi)
    ids_drivers = get_driver_id(taxi)
    post_code = london_address['PostCode']
    items_post_code = post_code.find()
    grids = [{'oseast1m': i['oseast1m'],'osnrth1m': i['osnrth1m']} for i in items_post_code]
    
    id_start = 0
    
    if orders.count_documents({}) != 0:
        id_start = items_orders.sort([("id", pymongo.DESCENDING)])[0]['id'] + 1
    package = []
    
    for i in range(n_count):
            
        client_id = random.choice(ids_clients)
        driver_id = random.choice(ids_drivers)
        ids_clients.remove(client_id)
        ids_drivers.remove(driver_id)
        price = random.randint(0, 1500)
        start = random.choice(grids)
        end = random.choice(grids)
        
        while end['oseast1m'] == start['oseast1m'] and end['osnrth1m'] == start['osnrth1m']:
            end = random.choice(grids)
            
        item = {"id": id_start, 
                "start": start, 
                "end": end, 
                "clientId": client_id, 
                "driverId": driver_id, 
                "price": price, 
                "closed": False}

        package.append(item)
        id_start += 1
        
    orders.insert_many(package)

In [10]:
def simulate_movement(taxi, london_address, n_count_order=1, n_count_worde = 100):
    movement = taxi['Movement']
    items_movement = movement.find()
    
    orders = taxi['Orders']
    items_orders = orders.find({"closed": False})
    ids_drivers_booked = [i['driverId'] for i in items_orders]
    
    ids_drivers_go = []
    n = n_count_order if len(ids_drivers_booked) > n_count_order else len(ids_driver_booked)
    tmp = ids_drivers_booked.copy()
    for i in range(n):
        id_dbd = random.choice(tmp)
        ids_drivers_go.append(id_dbd)
        tmp.remove(id_dbd)
    
 
    post_code = london_address['PostCode']
    items_post_code = post_code.find()
    grids = [{'oseast1m': i['oseast1m'],'osnrth1m': i['osnrth1m']} for i in items_post_code]

    ids_drivers = get_drivers_id(taxi)
    id_driver = random.choice(ids_drivers)
    
    id_start = 0
    
    if  movement.count_documents({}) != 0:
        id_start = items_movement.sort([("id", pymongo.DESCENDING)])[0]['id'] + 1
        
   
    if not(id_driver in ids_drivers_booked):
        for i in range(n_count_worde):
            position = random.choice(grids)
            item = {"id": id_start, 
                    "driverId": id_driver,
                    "position": position,
                    "booked": False}
            movement.insert_one(item)
            id_start += 1

    res = []
    for i in ids_drivers_go:
        tmp_grids = grids.copy()
        item_orders = orders.find({"driverId": i, "closed": False})[0]
        print(item_orders)
        position = random.choice(tmp_grids)
        item_m = {'id': id_start,
                  'driverId': i,
                  'position': position,
                  'booked': True}
        movement.insert_one(item_m)
        id_start += 1
        item_m['id'] = id_start
        item_m.pop('_id', None)
        print(item_m)
        movement.insert_one(item_m)
        while item_orders['end'] != position:
            tmp_grids.remove(position)
            position = random.choice(tmp_grids)
            item_m = {'id': id_start,
                      'driverId': i,
                      'position': position,
                      'booked': True}
#             movement.update_one({'id': id_start}, {'$set': {'position': position}})
        movement.update_one({'id': id_start}, {'$set': {'position': position}})
        id_start += 1
        item_m['id'] = id_start
        item_m['booked'] = False
        movement.insert_one(item_m)
        
        item_orders['closed'] = True
        orders.update_one({'id': item_orders['id']}, {'$set': {'closed': True}})
        res.append(item_orders.copy())
    return res

In [250]:
order = simulate_movement(client['Taxi'], client['LondonAddress'], n_count_worde=0)

{'_id': ObjectId('61b3e07c3de7cee630a743ca'), 'id': 85, 'start': {'oseast1m': 532038, 'osnrth1m': 190245}, 'end': {'oseast1m': 531300, 'osnrth1m': 180900}, 'clientId': 47, 'driverId': 122, 'price': 762, 'closed': False}
{'id': 122, 'driverId': 122, 'position': {'oseast1m': 533065, 'osnrth1m': 181488}, 'booked': True}


In [11]:
def create_feedback(taxi, order):
    feedback = taxi['Feedback']
    items_feedback = feedback.find()
    raiting = random.randint(1,6)
    message = ''.join(random.choices(string.ascii_letters + string.digits, k=random.randint(5,25)))
    id_start = 0
    
    if  feedback.count_documents({}) != 0:
        id_start = items_feedback.sort([("id", pymongo.DESCENDING)])[0]['id'] + 1
        
    item = {'id': id_start,
            'driverId': order['driverId'],
            'orderId': order['id'],
            'clientId': order['clientId'],
            'message': message,
            'raiting': raiting }
    feedback.insert_one(item)
    items_feedback_driver = feedback.find({'driverId': order['driverId']})
    avarega_raiting = 0
    n = 0
    for i in items_feedback_driver:
        avarega_raiting += i['raiting']
        n += 1
        
    avarega_raiting = round(avarega_raiting / n, 2)
    
    return avarega_raiting

In [267]:
new_rainting = create_feedback(client['Taxi'], order[0])

In [273]:
drivers = client['Taxi']['Driver']
drivers.update_one({'id': order[0]['driverId']}, {'$set': {'raiting': new_rainting}})

In [284]:
items = drivers.find({"raiting": {'$ne': None}})

In [285]:
items_sort = items.sort([("raiting", pymongo.DESCENDING)]).limit(100)

In [287]:
for i in items_sort:
    driver = i['id']
    raiting = i['raiting']
    print(f'id: {driver}, raiting: {raiting}')

id: 120, raiting: 4
id: 83, raiting: 3.83
id: 122, raiting: 3.2
id: 85, raiting: 3
